# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [4]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [5]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [7]:
data[['call', 'race']].head()

,call,race
0,0.0,w
1,0.0,w
2,0.0,b
3,0.0,b
4,0.0,w


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

**1. Answer**: The appropriate test for this problem is the check if the probability of getting call back is the significantly different between black and white sounding name. CLT is applied to it because the observations are independent random variables.

**2. Answer**: 
* Null hypothesis: The mean acceptance between white and black sounding name is not significantly different.
* Alternate hypothesis: The mean acceptance between white and black sounding name is significantly different.
              

In [140]:
w = np.array(data.call[data.race=='w'].values.tolist())
b = np.array(data.call[data.race=='b'].values.tolist())

In [172]:
print('################# boot straping method ###################### \n ')
# prove that the prob of hired difference is 0

np.random.seed(42)
rounds = 1000
w_bs = np.empty(rounds)
b_bs = np.empty(rounds)

for i in range(rounds):
    w_bs[i] = np.sum(np.random.choice(w, size=500, replace=False))
    b_bs[i] = np.sum(np.random.choice(b, size=500, replace=False))


diff = (w_bs - b_bs)/rounds
diff_mean = (w_bs.mean() - b_bs.mean())/rounds
diff_std = (np.sqrt(w_bs.std()**2 + b_bs.std()**2))/rounds
diff_CIup = np.percentile(diff, 97.5)
diff_CIlow = np.percentile(diff, 2.5)


z_scores = (0 - diff_mean)/diff_std
p_value_bs = stats.norm.sf(abs(z_scores))

z95 = 1.96
margin_bs = z95 * diff_std/np.sqrt(len(data))*100

print('The mean difference between black and white sounding name is {:.2f}'.format(diff_mean))
print('The margin of error is {:.4f}%.'.format(margin_bs))
print('The confidence interval for the difference is {:.3f} for upper and {:.3f} for lower.'.format(diff_CIup, diff_CIlow))
print('p-value for the null hypothesis is {:.4f}. \n'.format(p_value_bs))


print('################# frequentionist method ###################### \n ')
# prove that the probability is the same (the difference in probability is zero)

w_prob_mean = np.sum(w)/len(w)
b_prob_mean = np.sum(b)/len(b)
w_std = np.sqrt(w_prob*(1-w_prob)/len(w))
b_std = np.sqrt(b_prob*(1-b_prob)/len(b))

diff_mean = w_prob_mean - b_prob_mean
diff_std = np.sqrt(w_std**2 + b_std**2)

z95 = 1.96
upper_fs = diff_mean + z95*diff_std
lower_fs = diff_mean - z95*diff_std

z_scores = (0 - diff_mean)/diff_std
p_value_fs = stats.norm.sf(abs(z_scores))

margin_fs = z95 * diff_std/np.sqrt(len(data))*100

print('The mean difference between black and white sounding name probability is {:.2f}'.format(diff_mean))
print('The margin of error is {:.4f}%.'.format(margin_fs))
print('The confidence interval for the difference is {:.3f} for upper and {:.3f} for lower.'.format(upper_fs, lower_fs))
print('p-value for the null hypothesis is {:.4f}. \n'.format(p_value_fs))

################# boot straping method ###################### 
 
The mean difference between black and white sounding name is 0.02
The margin of error is 0.0208%.
The confidence interval for the difference is 0.030 for upper and 0.001 for lower.
p-value for the null hypothesis is 0.0149. 

################# frequentionist method ###################### 
 
The mean difference between black and white sounding name probability is 0.03
The margin of error is 0.0219%.
The confidence interval for the difference is 0.047 for upper and 0.017 for lower.
p-value for the null hypothesis is 0.0000. 



In [7]:
# Your solution to Q3 here

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

Write a story describing the statistical significance in the context or the original problem.
Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

**4. Answer**: Both of the statistical test from bootstrapping and frequentionist methods are very low (p<0.05), suggesting that the null hypothesis is rejected, i.e. the difference in the acceptance rate of white-sounding name is in fact significantly higher than the acceptance rate of black-sounding name.

**5. Answer**: No, not necessarily. There could be other factors that related to the call, such as education levels, ages, industries, and genders. To amend my analysis, we have to perform multivariate analysis to see how strongly correlated calls are to each of these variables.